In [59]:
import fitz
path = "C:/DAEN690/2A13.pdf"
pdf = fitz.open(path)
pdf.metadata

{'format': 'PDF 1.6',
 'title': '2A13 TCDS',
 'author': 'DEBORAH JONES',
 'subject': '',
 'keywords': 'Manual',
 'creator': 'Microsoft® Word 2016',
 'producer': 'Adobe Acrobat Pro DC (32-bit) 21 Paper Capture Plug-in',
 'creationDate': "D:20211005120659-04'00'",
 'modDate': "D:20211005121358-04'00'",
 'trapped': '',
 'encryption': None}

In [60]:
#pc = pdf.page_count
#print(pc)

In [61]:
# This function is to get all the [x0, y0, x1, y1, word, block_no, line_no, word_no] from the given TCDS file.

In [62]:
def get_all_text(data):
    pc = data.page_count
    word_box = []
    for i in range(0,pc):
        ct = data.load_page(i)
        gw = ct.get_text('words')
        word_box += gw
    
    return word_box

In [63]:
all_text = get_all_text(pdf)
print(all_text)

[(214.00999450683594, 56.68177795410156, 284.94512939453125, 70.47637939453125, 'DEPARTMENT', 63, 0, 0), (287.4251708984375, 56.68177795410156, 301.2994384765625, 70.47637939453125, 'OF', 63, 0, 1), (303.8292541503906, 56.68177795410156, 397.8815612792969, 70.47637939453125, 'TRANSPORTATION', 63, 0, 2), (208.85000610351562, 68.34175109863281, 256.3890686035156, 82.1363525390625, 'FEDERAL', 64, 0, 0), (258.8790588378906, 68.34175109863281, 309.74481201171875, 82.1363525390625, 'AVIATION', 64, 0, 1), (312.2646789550781, 68.34175109863281, 403.09002685546875, 82.1363525390625, 'ADMINISTRATION', 64, 0, 2), (504.2200012207031, 91.10301971435547, 524.1909790039062, 103.10901641845703, '2A13', 65, 0, 0), (480.7900085449219, 101.66301727294922, 512.758056640625, 113.66901397705078, 'Revision', 66, 0, 0), (515.1400146484375, 101.66301727294922, 524.2000122070312, 113.66901397705078, '63', 66, 0, 1), (458.4700012207031, 112.22301483154297, 477.6490173339844, 124.22901153564453, 'Piper', 67, 0, 0

In [64]:
# merge the words which have same block_no and line_no form all_text 

In [65]:
def word_line(in_put):
    index = 0
    word_list = []
    output = [[]]
    for i in range(0,len(in_put)-1):
        if(in_put[i][-1]+1 == in_put[i+1][-1]):
            word_list += [in_put[i][-4]]  
        else:
            if(in_put[i+1][-1]==0):
                word_list += [in_put[i][-4]]
                word_list = [' '.join(word_list)]
                word_list += [in_put[i][-3],in_put[i][-2],in_put[i][-1]]
                word_list.insert( 0, in_put[i-in_put[i][-1]][0])
                word_list.insert( 1, in_put[i-in_put[i][-1]][1])
                word_list.insert( 2, in_put[i][2])
                word_list.insert( 3, in_put[i][3])
            print(word_list)

            output[index] += word_list
            index += 1
            word_list = []
            output += [[]]
    
    return output

In [66]:
out = word_line(all_text)

[214.00999450683594, 56.68177795410156, 397.8815612792969, 70.47637939453125, 'DEPARTMENT OF TRANSPORTATION', 63, 0, 2]
[208.85000610351562, 68.34175109863281, 403.09002685546875, 82.1363525390625, 'FEDERAL AVIATION ADMINISTRATION', 64, 0, 2]
[504.2200012207031, 91.10301971435547, 524.1909790039062, 103.10901641845703, '2A13', 65, 0, 0]
[480.7900085449219, 101.66301727294922, 524.2000122070312, 113.66901397705078, 'Revision 63', 66, 0, 1]
[458.4700012207031, 112.22301483154297, 524.2150268554688, 124.22901153564453, 'Piper Aircraft, Inc', 67, 0, 2]
[390.19000244140625, 133.34300231933594, 430.4505920410156, 145.34901428222656, 'PA-28-140', 68, 0, 0]
[454.6300048828125, 133.34300231933594, 494.8905944824219, 145.34901428222656, 'PA-28-151', 68, 1, 0]
[390.19000244140625, 143.9029998779297, 430.4505920410156, 155.9090118408203, 'PA-28-150', 69, 0, 0]
[454.6300048828125, 143.9029998779297, 494.8905944824219, 155.9090118408203, 'PA-28-161', 69, 1, 0]
[390.19000244140625, 154.46299743652344

In [67]:
type(out)

list

In [68]:
# Get the X1 of first given word

In [69]:
def get_word_x1(df, word):
    
    for i in range(0,len(df)):
        if(df[i][4] == word):
            return df[i][2]

In [70]:

fuel_x1 = get_word_x1(out, 'Fuel')
print(fuel_x1)

106.07099914550781


In [71]:
# define the line is [key_term, Model_header, information] to [1,2,3]

In [72]:
def detect_khi(in_put):
    index = 0
    temp_list= []
    out_put = [[]]
    for i in range(0,len(in_put)-1):
        if(in_put[i][0] <= fuel_x1) & (in_put[i][-2] == 0) & (in_put[i][-1] <= 5 ):
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            temp_list.insert(8, 'key_term')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
        elif(in_put[i][0] <= fuel_x1) & (in_put[i][-2] == 0) & (in_put[i][4].find('Model') > 0):
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            temp_list.insert(8, 'header')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
        else:
            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
            temp_list.insert(8, 'info')
            out_put[index] = temp_list
            temp_list = []
            out_put += [[]]
            index += 1
    print(out_put)
    return out_put

In [73]:
#def detect_khi(in_put):
#    index = 0
#    temp_list= []
#    out_put = [[]]
#    for i in range(0,len(in_put)-1):
#        if(in_put[i][0] <= fuel_x1)& (in_put[i][-1] <= 5 ):
#            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
#            temp_list.insert(8, 'key_term')
#            out_put[index] = temp_list
#            temp_list = []
#            out_put += [[]]
#            index += 1
#        elif(in_put[i][0] <= fuel_x1) & (in_put[i][4].find('Model') > 0):
#            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
#            temp_list.insert(8, 'header')
#            out_put[index] = temp_list
#            temp_list = []
#            out_put += [[]]
#            index += 1
#        else:
#            temp_list += [in_put[i][0], in_put[i][1], in_put[i][2], in_put[i][3], in_put[i][4], in_put[i][5], in_put[i][6], in_put[i][7]]
#            temp_list.insert(8, 'info')
#            out_put[index] = temp_list
#            temp_list = []
#            out_put += [[]]
#            index += 1
#    print(out_put)
#    return out_put

In [74]:
kt = detect_khi(out)
kt

[[214.00999450683594, 56.68177795410156, 397.8815612792969, 70.47637939453125, 'DEPARTMENT OF TRANSPORTATION', 63, 0, 2, 'info'], [208.85000610351562, 68.34175109863281, 403.09002685546875, 82.1363525390625, 'FEDERAL AVIATION ADMINISTRATION', 64, 0, 2, 'info'], [504.2200012207031, 91.10301971435547, 524.1909790039062, 103.10901641845703, '2A13', 65, 0, 0, 'info'], [480.7900085449219, 101.66301727294922, 524.2000122070312, 113.66901397705078, 'Revision 63', 66, 0, 1, 'info'], [458.4700012207031, 112.22301483154297, 524.2150268554688, 124.22901153564453, 'Piper Aircraft, Inc', 67, 0, 2, 'info'], [390.19000244140625, 133.34300231933594, 430.4505920410156, 145.34901428222656, 'PA-28-140', 68, 0, 0, 'info'], [454.6300048828125, 133.34300231933594, 494.8905944824219, 145.34901428222656, 'PA-28-151', 68, 1, 0, 'info'], [390.19000244140625, 143.9029998779297, 430.4505920410156, 155.9090118408203, 'PA-28-150', 69, 0, 0, 'info'], [454.6300048828125, 143.9029998779297, 494.8905944824219, 155.9090

[[214.00999450683594,
  56.68177795410156,
  397.8815612792969,
  70.47637939453125,
  'DEPARTMENT OF TRANSPORTATION',
  63,
  0,
  2,
  'info'],
 [208.85000610351562,
  68.34175109863281,
  403.09002685546875,
  82.1363525390625,
  'FEDERAL AVIATION ADMINISTRATION',
  64,
  0,
  2,
  'info'],
 [504.2200012207031,
  91.10301971435547,
  524.1909790039062,
  103.10901641845703,
  '2A13',
  65,
  0,
  0,
  'info'],
 [480.7900085449219,
  101.66301727294922,
  524.2000122070312,
  113.66901397705078,
  'Revision 63',
  66,
  0,
  1,
  'info'],
 [458.4700012207031,
  112.22301483154297,
  524.2150268554688,
  124.22901153564453,
  'Piper Aircraft, Inc',
  67,
  0,
  2,
  'info'],
 [390.19000244140625,
  133.34300231933594,
  430.4505920410156,
  145.34901428222656,
  'PA-28-140',
  68,
  0,
  0,
  'info'],
 [454.6300048828125,
  133.34300231933594,
  494.8905944824219,
  145.34901428222656,
  'PA-28-151',
  68,
  1,
  0,
  'info'],
 [390.19000244140625,
  143.9029998779297,
  430.450592041

In [75]:
for i in range(0, len(kt)):
    print(kt[i])

[214.00999450683594, 56.68177795410156, 397.8815612792969, 70.47637939453125, 'DEPARTMENT OF TRANSPORTATION', 63, 0, 2, 'info']
[208.85000610351562, 68.34175109863281, 403.09002685546875, 82.1363525390625, 'FEDERAL AVIATION ADMINISTRATION', 64, 0, 2, 'info']
[504.2200012207031, 91.10301971435547, 524.1909790039062, 103.10901641845703, '2A13', 65, 0, 0, 'info']
[480.7900085449219, 101.66301727294922, 524.2000122070312, 113.66901397705078, 'Revision 63', 66, 0, 1, 'info']
[458.4700012207031, 112.22301483154297, 524.2150268554688, 124.22901153564453, 'Piper Aircraft, Inc', 67, 0, 2, 'info']
[390.19000244140625, 133.34300231933594, 430.4505920410156, 145.34901428222656, 'PA-28-140', 68, 0, 0, 'info']
[454.6300048828125, 133.34300231933594, 494.8905944824219, 145.34901428222656, 'PA-28-151', 68, 1, 0, 'info']
[390.19000244140625, 143.9029998779297, 430.4505920410156, 155.9090118408203, 'PA-28-150', 69, 0, 0, 'info']
[454.6300048828125, 143.9029998779297, 494.8905944824219, 155.9090118408203

In [76]:
# sincet the detect_khi function can only define the first line of the headers.
# This function will mark all the other lines from 'info' to 'header'

In [77]:
def block_header(in_put):
    for i in range(0, len(in_put)-5):
        if(in_put[i][8] == 'header') & (in_put[i][5] == in_put[i+1][5]):
            in_put[i+1][8] = 'header'
            

In [78]:
bkt = block_header(kt)

In [79]:
# print all the words which have been marked as 'key_term'

In [82]:
for i in range(0, len(kt)-1):
    if(kt[i][-1] == 'key_term'):
        print(kt[i][-5])

Type Certificate Holder
Type Certificate Holder Record
Engine
Fuel
Engine Limits
Propeller and Propeller Limits
Propeller Spinner
Airspeed Limits
Page No.
Revision No.
Page No.
Revision No.
Page No.
Revision No.
2A13
Center of Gravity Range
Empty Weight C. G. Range
Maximum Weight
No. of Seats
Maximum Baggage
Fuel Capacity
Oil Capacity
Control Surface Movements
Nose Wheel Travel
Manufacturer's Serial Nos.
Engine
Fuel
Engine Limits
Propeller and Propeller Limits
Propeller Spinner
Airspeed Limits
Center of Gravity Range
Empty Wt. C. G. Range
Maximum Weight
No. of Seats
Maximum Baggage
Fuel Capacity
Oil Capacity
Control Surface Movements
Nose Wheel Travel
Manufacturer's Serial Nos.
Engine
Fuel
Engine Limits
Propeller and Propeller Limits
2A13
Propeller Spinner
Airspeed Limits
Center of Gravity Range
Empty Weight C. G. Range
Maximum Weight
No. of Seats
Maximum Baggage
Fuel Capacity
Oil Capacity
Control Surface Movements
Nose Wheel Travel
Manufacturer's Serial Nos.
Engine
Fuel
Engine Limits


In [81]:
for i in range(0, len(kt)-1):
    if(kt[i][-1] == 'key_term'):
        print(kt[i][-5])

Type Certificate Holder
Type Certificate Holder Record
Engine
Fuel
Engine Limits
Propeller and Propeller Limits
Propeller Spinner
Airspeed Limits
Page No.
Revision No.
Page No.
Revision No.
Page No.
Revision No.
2A13
Center of Gravity Range
Empty Weight C. G. Range
Maximum Weight
No. of Seats
Maximum Baggage
Fuel Capacity
Oil Capacity
Control Surface Movements
Nose Wheel Travel
Manufacturer's Serial Nos.
Engine
Fuel
Engine Limits
Propeller and Propeller Limits
Propeller Spinner
Airspeed Limits
Center of Gravity Range
Empty Wt. C. G. Range
Maximum Weight
No. of Seats
Maximum Baggage
Fuel Capacity
Oil Capacity
Control Surface Movements
Nose Wheel Travel
Manufacturer's Serial Nos.
Engine
Fuel
Engine Limits
Propeller and Propeller Limits
2A13
Propeller Spinner
Airspeed Limits
Center of Gravity Range
Empty Weight C. G. Range
Maximum Weight
No. of Seats
Maximum Baggage
Fuel Capacity
Oil Capacity
Control Surface Movements
Nose Wheel Travel
Manufacturer's Serial Nos.
Engine
Fuel
Engine Limits


In [57]:
# print all the words which have been marked as 'header'

In [58]:
for i in range(0, len(kt)-1):
    if(kt[i][-1] == 'header'):
        print(kt[i][-5])

II - Model PA-16S; 3 PCSM (Normal Category Only); Approved July 20, 1949


In [23]:
# The coming function is tried to add page into the df. 
# There's some error of the function and the page no is maybe not necessary.

In [59]:
#def add_page(df):
#    p = 1
#    for i in range(0,len(df)-1):
#        if(df[i][5] <= df[i+1][5]):
#            df.insert(-1, p)
#        else:
#            p += 1
#            df.insert(-1, p)
#            print(df)

In [66]:
#add_page(out)